In [17]:
import time
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import urllib
import re
from tqdm import tqdm

# 제품별 정보 페이지 url 패턴 docID 크롤링

In [73]:
list_prod_docID = []    # -> 제품페이지 url 패턴에 사용 (가나다순)

for page in range(1,37):
    url = f'https://terms.naver.com/list.naver?cid=42726&categoryId=58635&so=st3.asc&viewType=&categoryType=&page={page}' 
    resp = requests.get(url)
    html = BeautifulSoup(resp.text,'html.parser')
    
    #차단 막는 코드. 랜덤으로 시간차 준다.
    seed = np.random.randint(100)
    np.random.seed(seed)  
    a = np.random.randint(5)
    time.sleep(a)
    
    for index in range(1, 30, 2):
        list_prod_docID.append(html.select('strong>a')[index]['href'][19:26])    # 제품별 docID (url)
    print(f'{page}page crwaling')

# 마지막 37페이지 제품개수 적어서 [index] out of range 라서 따로 함
url = 'https://terms.naver.com/list.naver?cid=42726&categoryId=58635&so=st3.asc&viewType=&categoryType=&page=37' 
resp = requests.get(url)
html = BeautifulSoup(resp.text,'html.parser')
for index in range(1, 16, 2):
    list_prod_docID.append(html.select('strong>a')[index]['href'][19:26])
print('37page 완료')

1page crwaling
2page crwaling
3page crwaling
4page crwaling
5page crwaling
6page crwaling
7page crwaling
8page crwaling
9page crwaling
10page crwaling
11page crwaling
12page crwaling
13page crwaling
14page crwaling
15page crwaling
16page crwaling
17page crwaling
18page crwaling
19page crwaling
20page crwaling
21page crwaling
22page crwaling
23page crwaling
24page crwaling
25page crwaling
26page crwaling
27page crwaling
28page crwaling
29page crwaling
30page crwaling
31page crwaling
32page crwaling
33page crwaling
34page crwaling
35page crwaling
36page crwaling
37page 완료


# 제품별 정보 크롤링

In [75]:
list_info = []    # 제품정보담기위한 리스트

for index in tqdm(list_prod_docID):
    
    url = f'https://terms.naver.com/entry.naver?docId={index}&cid=42726&categoryId=58635'
    resp2 = requests.get(url)
    html2 = BeautifulSoup(resp2.text,'html.parser')

    for item in (html2.select('tbody td')[0:7]):
        list_info.append(re.sub(r'[" "\n]', "", item.text))
    
    #차단 막는 코드. 랜덤으로 시간차 준다.
    seed = np.random.randint(100)
    np.random.seed(seed)  
    a = np.random.randint(5)
    time.sleep(a)

# dataframe에 담기
df_naver_wiki_prod_info = pd.DataFrame(data = np.array(list_info).reshape(-1,7)
                                 , columns = ['제품명', '주종', '도수', '용량', '가격', '원재료', '제조사'])


100%|████████████████████████████████████████████████████████████████████████████████| 548/548 [22:59<00:00,  2.52s/it]


# 일부 전처리

In [76]:
df_naver_wiki_prod_info

,제품명,주종,도수,용량,가격,원재료,제조사
0,가덕순쌀막걸리,탁주,6%,750ml,"￦13,500(750ml*9병)(가격은판매처별로상이할수있습니다)","국내산쌀,효모,조효소재,물엿,아스파탐,정제수",농업회사법인명세주가(주)
1,가을빛레드와인,과실주,14%,750ml,"￦35,000(가격은판매처별로상이할수있습니다)",포도(캠벨얼리),예인화원
2,가평잣막걸리,탁주,6%,750ml,"￦1,450(가격은판매처별로상이할수있습니다.)","쌀,정제수,쌀입국,잣,효모,정제효소등",가평우리술
3,감보드카,일반증류주,32%,375ml,"￦14,000(가격은판매처별로상이할수있습니다)",감(국내산)100%,양촌감영농조합법인
4,감그린레귤러,과실주,12%,750ml,"￦18,000(가격은판매처별로상이할수있습니다.)",감(국내산),청도감와인(주)
...,...,...,...,...,...,...,...
543,33JU,일반증류주,18%,500ml/200ml,"￦49,000/￦23,000(가격은판매처별로상이할수있습니다)","산양산삼,주정,결정과당,효소처리스테비아,에리스리톨,호박산,꿀",농업회사법인33가주식회사
544,52C,일반증류주,17.50%,500ml,"￦10,000(가격은판매처별로상이할수있습니다)","정제수,주정,쌀증류식소주원액,오이",농업회사법인(주)청산녹수
545,DMZ240km_450ml,기타주류,25%,450ml,가격은판매처별로상이할수있습니다.,"자연송이버섯,브랜디원액,블랙커런트,벌꿀",솔래원
546,G12골디락스,탁주,12%,500ml,"￦12,000(가격은판매처별로상이할수있습니다)","정제수,멥쌀(국내산),찹쌀(국내산),누룩,종국,효모,밀함유",농업회사법인(주)청산녹수


In [77]:
df_naver_wiki_prod_info['가격'] = df_naver_wiki_prod_info['가격'].apply(lambda x : re.sub(r'[￦,.()가격은판매처별로상이할수있습니다]', "", x))
df_naver_wiki_prod_info

,제품명,주종,도수,용량,가격,원재료,제조사
0,가덕순쌀막걸리,탁주,6%,750ml,13500750ml*9병,"국내산쌀,효모,조효소재,물엿,아스파탐,정제수",농업회사법인명세주가(주)
1,가을빛레드와인,과실주,14%,750ml,35000,포도(캠벨얼리),예인화원
2,가평잣막걸리,탁주,6%,750ml,1450,"쌀,정제수,쌀입국,잣,효모,정제효소등",가평우리술
3,감보드카,일반증류주,32%,375ml,14000,감(국내산)100%,양촌감영농조합법인
4,감그린레귤러,과실주,12%,750ml,18000,감(국내산),청도감와인(주)
...,...,...,...,...,...,...,...
543,33JU,일반증류주,18%,500ml/200ml,49000/23000,"산양산삼,주정,결정과당,효소처리스테비아,에리스리톨,호박산,꿀",농업회사법인33가주식회사
544,52C,일반증류주,17.50%,500ml,10000,"정제수,주정,쌀증류식소주원액,오이",농업회사법인(주)청산녹수
545,DMZ240km_450ml,기타주류,25%,450ml,,"자연송이버섯,브랜디원액,블랙커런트,벌꿀",솔래원
546,G12골디락스,탁주,12%,500ml,12000,"정제수,멥쌀(국내산),찹쌀(국내산),누룩,종국,효모,밀함유",농업회사법인(주)청산녹수


In [78]:
# 파일저장
df_naver_wiki_prod_info.to_csv('naver_wiki_prod_info.csv')
df_naver_wiki_prod_info.to_excel('naver_wiki_prod_info.xlsx')